In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re
from sklearn.metrics.pairwise import cosine_similarity  # Correct import
import numpy as np


In [131]:

# Đọc dữ liệu từ tập CSV vào DataFrame
df = pd.read_csv('DATADULICHQUYNHON.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df


,Name_Hotel,original_price_list,Link,star_rating_hotel_list,Location,Convenient,Aria_locations,Comment_Hotel,User_vote_hotel,Review_Hotel,hightlight_Hotel
0,Grand Hyams Hotel - Quy Nhon Beach,3950100,https://www.booking.com/hotel/vn/grand-hyams-q...,5.0,"28 Nguyen Hue, Le Loi ward, Quy Nhon city, Bin...","Hồ bơi trong nhà, WiFi miễn phí, Phòng gia ...","Tượng Đài Chiến Thắng: 150 m, Công Viên Bàu Se...","Tram(Việt Nam): “Sang trọng, yên tĩnh, các bạ...",9.3,"Nằm ở Quy Nhơn, cách Quy Nhon Beach 2 phút đi ...","Nhìn ra biển,Nhìn ra địa danh nổi tiếng,Hồ bơi..."
1,FLC City Hotel Beach Quy Nhon,3937950,https://www.booking.com/hotel/vn/flc-beach-cit...,5.0,"An Duong Vuong Road Quy Nhon City, Binh Dinh P...","Hồ bơi trong nhà, Giáp biển, WiFi miễn phí, ...","Công Viên: 300 m, Đu Quay Uốn Lượn: 400 m, Côn...","Huy(Việt Nam): “Phòng rộng, view đẹp, vị trí ...",8.6,"Nằm giáp biển, FLC City Hotel Beach Quy Nhon c...","Nhìn ra biển, Địa điểm hàng đầu: Được khách gầ..."
2,RAON Hotel - STAY 24H,1426513,https://www.booking.com/hotel/vn/nam-thu.vi.ht...,4.0,"09 Nguyễn Thị Định, Quy Nhơn, Việt Nam","Hồ bơi ngoài trời, Giáp biển, WiFi miễn phí,...","Công Viên: 500 m, Công Viên: 600 m, Đu Quay Uố...","Gabriel(Việt Nam): “Mọi thứ đều ổn, sạch sẽ, ...",9.3,"Tọa lạc ở Quy Nhơn, cách Quy Nhon Beach 7 phút...","Hồ bơi có tầm nhìn,Hướng nhìn ra đường phố yên..."
3,MENTO HOTEL QUY NHƠN,1863000,https://www.booking.com/hotel/vn/mento-quy-nho...,3.0,"Nguyễn Huệ 409C - 409D, Quy Nhơn, Việt Nam","3 hồ bơi, Giáp biển, WiFi miễn phí, Xe đưa đ...","Ngã 6 Ngô Mây: 250 m, Công Viên: 400 m, Đu Qua...",No comment,8.4,"Nằm giáp biển, MENTO HOTEL QUY NHƠN cung cấp c...","Nhìn ra thành phố,Nhìn ra núi,Nhìn ra địa danh..."
4,Moon's House Homestay Quy Nhơn,1050000,https://www.booking.com/hotel/vn/moon-39-s-hou...,2.0,"82 Vũ Bảo, Quy Nhơn, Việt Nam","WiFi miễn phí, Xe đưa đón sân bay, Phòng g...","Ngã 6 Ngô Mây: 350 m, Công Viên: 550 m, Đu Qua...","Vy(Việt Nam): “Không gian vừa đủ, sạch sẽ, mề...",9.5,Nằm ở Quy Nhơn và cách Quy Nhon Beach chưa đến...,"Sân hiên, Địa điểm hàng đầu: Được khách gần đâ..."
...,...,...,...,...,...,...,...,...,...,...,...
438,Q House- Green Villa Quy Nhơn,8967240,https://www.booking.com/hotel/vn/q-house-homes...,0.0,"70/14/12 70/14/13 Han Mac Tu, Ghenh Rang Ward,...","WiFi miễn phí, Xe đưa đón sân bay, Phòng g...","Công Viên: 600 m, Công Viên: 1,1 km, Đu Quay U...",No comment,9.0,"Cách Quy Nhon Beach 4 phút đi bộ, Q House- Gre...","Nhìn ra thành phố,Sân hiên,Nhìn ra vườn,Hướng ..."
439,Căn hộ du lịch Quy Nhơn giá rẻ-view biển 7,2904000,https://www.booking.com/hotel/vn/quy-nhon-colo...,0.0,"Chung cư An Phú Thịnh Garden Tower, Khu đô thị...","WiFi miễn phí, Chỗ đỗ xe miễn phí","Công Viên Đống Đa: 2,7 km, Công Viên Đống Đa: ...",No comment,8.4,"Tọa lạc ở Quy Nhơn thuộc Bình Định, Căn hộ du ...",Sân hiênNo hightlight
440,Quy Nhon Homestay BnB,8997000,https://www.booking.com/hotel/vn/quynhonhomest...,0.0,"20 Trần Kỳ Phong, Quy Nhơn, Bình Định, Quy Nhơ...","Giáp biển, WiFi miễn phí, Xe đưa đón sân ba...","Công Viên: 500 m, Công Viên: 900 m, Đu Quay Uố...",Hồng(Việt Nam): “Hàng xóm thân thiện! Địa điể...,8.4,Nằm ở Quy Nhơn và cách Quy Nhon Beach 6 phút đ...,"Nhìn ra thành phố,Nhìn ra núi,Sân hiên,Hướng n..."
441,CĂN HỘ QUY NHƠN,5400000,https://www.booking.com/hotel/vn/can-ho-quy-nh...,0.0,"76 Trần Hưng Đạo 26, Quy Nhơn, Việt Nam","Hồ bơi, WiFi miễn phí, Chỗ đỗ xe miễn phí","Tượng Đài Chiến Thắng: 750 m, Công Viên Đống Đ...",No comment,8.4,No review,Sân hiênNo hightlight


In [132]:
df.columns

Index(['Name_Hotel', 'original_price_list', 'Link', 'star_rating_hotel_list',
       'Location', 'Convenient', 'Aria_locations', 'Comment_Hotel',
       'User_vote_hotel', 'Review_Hotel', 'hightlight_Hotel'],
      dtype='object')

In [133]:
df.isnull().sum()

Name_Hotel                0
original_price_list       0
Link                      0
star_rating_hotel_list    0
Location                  0
Convenient                0
Aria_locations            0
Comment_Hotel             0
User_vote_hotel           0
Review_Hotel              0
hightlight_Hotel          0
dtype: int64

# Không dùng PCA

In [147]:
# Xử lý dữ liệu, nếu cần
# Ví dụ: Kết hợp các trường 'Tên khách sạn' và 'Tiện ích' thành một trường duy nhất 'Mô tả'
#df['Mô tả'] = df['Name_Hotel'] + ' ' + df['star_rating_hotel_list'] + ' ' + df['voting_user_list'] + ' ' + df['review_list']
#numeric_columns = ['star_rating_hotel_list', 'User_vote_hotel','original_price_list','Location', 'Convenient', 'Aria_locations', 'Comment_Hotel','Review_Hotel', 'hightlight_Hotel']

# Tạo cột 'description' chỉ từ các cột giá trị số
df['description'] = (
    
    df['Location'].astype(str)+
    ' ' +
    df['Convenient'].astype(str)+
    ' ' +
    df['Aria_locations'].astype(str)+
    ' ' +
    df['Comment_Hotel'].astype(str)+
    ' ' +
    df['Review_Hotel'].astype(str)+
    ' ' +
    df['hightlight_Hotel'].astype(str)
)
# Sử dụng TF-IDF để biểu diễn văn bản thành ma trận số
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description'].fillna(''))

# Tính cosine similarity giữa các khách sạn
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# Hàm đề xuất dựa trên cosine similarity
def content_based_recommendation(hotel_name, cosine_similarities, df):
    hotel_index = df[df['Name_Hotel'] == hotel_name].index
    if not hotel_index.empty:
        hotel_index = hotel_index[0]
        similar_hotels = list(enumerate(cosine_similarities[hotel_index]))
        similar_hotels = sorted(similar_hotels, key=lambda x: x[1], reverse=True)
        similar_hotels = similar_hotels[1:11]  # Lấy 10 khách sạn tương tự

        # Lấy tất cả giá trị từ danh sách similar_hotels
        recommended_hotels = [df.iloc[hotel[0]]['Name_Hotel'] for hotel in similar_hotels]
        return recommended_hotels
    else:
        print(f'Hotel not found: {hotel_name}')
        return []

# Đề xuất 10 khách sạn tương tự với khách sạn có tên là 'Tên khách sạn cần đề xuất'
recommended_hotels = content_based_recommendation("Grand Hyams Hotel - Quy Nhon Beach", cosine_similarities, df)
print(recommended_hotels)

['FLC City Hotel Beach Quy Nhon', 'Pearl Beach Hotel Quy Nhon', 'Fleur de Lys Hotel Quy Nhon', 'Anya Hotel Quy Nhon', 'Grand Hyams Hotel - TMS Tầng 21 View Biển Quy Nhơn - 2PN', 'Khách Sạn Thành Nhân', 'CALI HOTEL Quy Nhon', 'MENTO HOTEL QUY NHƠN', 'Anantara Quy Nhon Villas', 'Thanh Van Hotel Quy Nhon']


In [148]:
print(cosine_similarities)
cosine_similarities.shape

[[1.         0.54767981 0.41389345 ... 0.39418713 0.25257051 0.38137306]
 [0.54767981 1.         0.39242016 ... 0.38773626 0.21994392 0.3679579 ]
 [0.41389345 0.39242016 1.         ... 0.41176207 0.21369565 0.3239924 ]
 ...
 [0.39418713 0.38773626 0.41176207 ... 1.         0.31382945 0.44201797]
 [0.25257051 0.21994392 0.21369565 ... 0.31382945 1.         0.67317604]
 [0.38137306 0.3679579  0.3239924  ... 0.44201797 0.67317604 1.        ]]


(443, 443)

In [149]:
for i in range(3):
    print(result_df['Information'][i])

FLC City Hotel Beach Quy Nhon: {'Name_Hotel': 'FLC City Hotel Beach Quy Nhon', 'original_price_list': 3937950, 'Link': 'https://www.booking.com/hotel/vn/flc-beach-city-quy-nhon.vi.html?label=gen173nr-1FCAEoggI46AdIM1gEaPQBiAEBmAEquAEXyAEM2AEB6AEB-AELiAIBqAIDuAKutIavBsACAdICJDgxNmM1ODQyLWRiNGMtNGZhZC1iYzkxLTg2MDM3MmM4NDE4NNgCBuACAQ&sid=eceff66f0c492ef7b95dec3d8b86b680&aid=304142&ucfs=1&arphpl=1&checkin=2024-03-13&checkout=2024-03-16&dest_id=-3727579&dest_type=city&group_adults=1&req_adults=1&no_rooms=1&group_children=1&req_children=1&age=7&req_age=7&hpos=2&hapos=2&sr_order=popularity&srpvid=168041259af5048a&srepoch=1709284560&all_sr_blocks=759894504_333145989_0_1_0&highlighted_blocks=759894504_333145989_0_1_0&matching_block_id=759894504_333145989_0_1_0&sr_pri_blocks=759894504_333145989_0_1_0__393795000&from=searchresults#hotelTmpl', 'star_rating_hotel_list': 5.0, 'Location': 'An Duong Vuong Road Quy Nhon City, Binh Dinh Province, Quy Nhơn, Việt Nam', 'Convenient': 'Hồ bơi trong nhà, Gi

In [150]:
import pandas as pd
hotels_to_check = recommended_hotels

# Create an empty DataFrame
result_df = pd.DataFrame(columns=['Hotel', 'Target Hotel', 'Similarity Score', 'Information'])

# Target hotel for comparison
target_hotel = "Grand Hyams Hotel - Quy Nhon Beach"

# Check the similarities between the specified hotels and the target hotel
for hotel_name in hotels_to_check:
    # Check if both hotels are in the DataFrame
    if hotel_name in df['Name_Hotel'].values and target_hotel in df['Name_Hotel'].values:
        # Calculate the cosine similarity between the two hotels
        hotel_index = df[df['Name_Hotel'] == hotel_name].index[0]
        target_index = df[df['Name_Hotel'] == target_hotel].index[0]
        similarity_score = cosine_similarities[hotel_index, target_index]

        # Get the information for each hotel
        hotel_info = df.iloc[hotel_index].to_dict()
        target_info = df.iloc[target_index].to_dict()

        # Append information to the result DataFrame
        result_df = result_df.append({
            'Hotel': hotel_name,
            'Target Hotel': target_hotel,
            'Similarity Score': similarity_score,
            'Information': f"{hotel_name}: {hotel_info}\n{target_hotel}: {target_info}"
        }, ignore_index=True)
    else:
        result_df = result_df.append({
            'Hotel': hotel_name,
            'Target Hotel': target_hotel,
            'Similarity Score': 'Not found',
            'Information': 'At least one of the hotels not found'
        }, ignore_index=True)

# Display the result DataFrame
result_df


,Hotel,Target Hotel,Similarity Score,Information
0,FLC City Hotel Beach Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.547680,FLC City Hotel Beach Quy Nhon: {'Name_Hotel': ...
1,Pearl Beach Hotel Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.536724,Pearl Beach Hotel Quy Nhon: {'Name_Hotel': 'Pe...
2,Fleur de Lys Hotel Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.532963,Fleur de Lys Hotel Quy Nhon: {'Name_Hotel': 'F...
3,Anya Hotel Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.527037,Anya Hotel Quy Nhon: {'Name_Hotel': 'Anya Hote...
4,Grand Hyams Hotel - TMS Tầng 21 View Biển Quy ...,Grand Hyams Hotel - Quy Nhon Beach,0.523870,Grand Hyams Hotel - TMS Tầng 21 View Biển Quy ...
5,Khách Sạn Thành Nhân,Grand Hyams Hotel - Quy Nhon Beach,0.517479,Khách Sạn Thành Nhân: {'Name_Hotel': 'Khách Sạ...
6,CALI HOTEL Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.517219,CALI HOTEL Quy Nhon: {'Name_Hotel': 'CALI HOTE...
7,MENTO HOTEL QUY NHƠN,Grand Hyams Hotel - Quy Nhon Beach,0.513754,MENTO HOTEL QUY NHƠN: {'Name_Hotel': 'MENTO HO...
8,Anantara Quy Nhon Villas,Grand Hyams Hotel - Quy Nhon Beach,0.511061,Anantara Quy Nhon Villas: {'Name_Hotel': 'Anan...
9,Thanh Van Hotel Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.507020,Thanh Van Hotel Quy Nhon: {'Name_Hotel': 'Than...


# 3 Field numeric và 6 field Obj

In [128]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

# Assume df is your DataFrame containing the columns 'star_rating_hotel_list', 'original_price_list', 'User_vote_hotel', and 'Name_Hotel'

df['description'] = (
    # df['star_rating_hotel_list'].astype(str) +
    # ' ' +
    # df['original_price_list'].astype(str) +
    # ' ' +
    # df['User_vote_hotel'].astype(str)+
    # ' ' +
    df['Location'].astype(str)+
    ' ' +
    df['Convenient'].astype(str)+
    ' ' +
    df['Aria_locations'].astype(str)+
    ' ' +
    df['Comment_Hotel'].astype(str)+
    ' ' +
    df['Review_Hotel'].astype(str)+
    ' ' +
    df['hightlight_Hotel'].astype(str)
)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description'])

# Sử dụng PCA để giảm chiều dữ liệu
# num_components = 2
# pca = PCA(n_components=num_components)
# pca_result = pca.fit_transform(tfidf_matrix.toarray())
scaler_star_rating = StandardScaler()
df['star_rating_hotel_list_normalized'] = scaler_star_rating.fit_transform(df[['star_rating_hotel_list']])

# Chuẩn hóa cột 'original_price_list'
scaler_original_price = StandardScaler()
df['original_price_list_normalized'] = scaler_original_price.fit_transform(df[['original_price_list']])

# Chuẩn hóa cột 'User_vote_hotel'
scaler_user_vote = StandardScaler()
df['User_vote_hotel_normalized'] = scaler_user_vote.fit_transform(df[['User_vote_hotel']])

# scaler_user_vote = StandardScaler()
# df['Location_normalized'] = scaler_user_vote.fit_transform(df[['Location']])
# scaler_user_vote = StandardScaler()
# df['Convenient_normalized'] = scaler_user_vote.fit_transform(df[['Convenient']])
# scaler_user_vote = StandardScaler()
# df['Aria_locations_normalized'] = scaler_user_vote.fit_transform(df[['Aria_locations']])
# scaler_user_vote = StandardScaler()
# df['Comment_Hotel_normalized'] = scaler_user_vote.fit_transform(df[['Comment_Hotel']])
# scaler_user_vote = StandardScaler()
# df['Review_Hotel_normalized'] = scaler_user_vote.fit_transform(df[['Review_Hotel']])
# scaler_user_vote = StandardScaler()
# df['hightlight_Hotel_normalized'] = scaler_user_vote.fit_transform(df[['hightlight_Hotel']])
# # Chuyển đổi thành dạng One-Hot Encoding
df = pd.get_dummies(df, columns=['star_rating_hotel_list', 'original_price_list', 'User_vote_hotel'])

columns_for_pca = ['star_rating_hotel_list_normalized', 'original_price_list_normalized', 'User_vote_hotel_normalized']


# # Sử dụng PCA để giảm chiều dữ liệu
num_components = 2
pca = PCA(n_components=num_components)
pca_result = pca.fit_transform(df[columns_for_pca])

combined_features = np.hstack((pca_result, tfidf_matrix.toarray()))

# Tính độ tương tự cosine giữa khách sạn mẫu và tất cả các khách sạn
similarity_scores = cosine_similarity(combined_features)
# Sử dụng TF-IDF để chuyển đổi văn bản thành vector
def content_based_recommendation(hotel_name, similarity_scores, df):
    hotel_index = df[df['Name_Hotel'] == hotel_name].index
    if not hotel_index.empty:
        hotel_index = hotel_index[0]
        similar_hotels = list(enumerate(similarity_scores[hotel_index]))
        similar_hotels = sorted(similar_hotels, key=lambda x: x[1], reverse=True)
        similar_hotels = similar_hotels[1:11]  # Lấy 10 khách sạn tương tự

        # Lấy tất cả giá trị từ danh sách similar_hotels
        recommended_hotels = [df.iloc[hotel[0]]['Name_Hotel'] for hotel in similar_hotels]
        return recommended_hotels
    else:
        print(f'Hotel not found: {hotel_name}')
        return []

# Đề xuất 10 khách sạn tương tự với khách sạn có tên là 'Tên khách sạn cần đề xuất'
recommended_hotels = content_based_recommendation("Grand Hyams Hotel - Quy Nhon Beach", similarity_scores, df)
print(recommended_hotels)



['TMS Pullman Căn Hộ Cao Cấp Quy Nhơn', 'FLC City Hotel Beach Quy Nhon', 'FLC Sea Tower Quy Nhơn- Đông Ngô House', 'Seaside Boutique Resort Quy Nhon', 'TMS Pullman Quy Nhơn The Maison House', 'FLC Sea Tower Qui Nhơn Apartment - Căn Hộ Sea View', "L'amor Boutique Hotel Quy Nhơn", 'Armin Homes Altara Residences Quy Nhon', 'Anya Premier Hotel Quy Nhon', 'TMS Luxury Condotel Quy Nhơn - Ban Công View Biển']


In [129]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
hotels_to_check = recommended_hotels


# Create an empty DataFrame
result_df = pd.DataFrame(columns=['Hotel', 'Target Hotel', 'Similarity Score', 'Information'])

# Target hotel for comparison
target_hotel = "Grand Hyams Hotel - Quy Nhon Beach"

# Check the similarities between the specified hotels and the target hotel
for hotel_name in hotels_to_check:
    # Check if both hotels are in the DataFrame
    if hotel_name in df['Name_Hotel'].values and target_hotel in df['Name_Hotel'].values:
        # Calculate the cosine similarity between the two hotels
        hotel_index = df[df['Name_Hotel'] == hotel_name].index[0]
        target_index = df[df['Name_Hotel'] == target_hotel].index[0]
        similarity_score = cosine_similarity(pca_result[hotel_index].reshape(1, -1), pca_result[target_index].reshape(1, -1))[0][0]

        # Get the information for each hotel
        hotel_info = df.iloc[hotel_index].to_dict()
        target_info = df.iloc[target_index].to_dict()

        # Append information to the result DataFrame
        result_df = result_df.append({
            'Hotel': hotel_name,
            'Target Hotel': target_hotel,
            'Similarity Score': similarity_score,
            'Information': f"{hotel_name}: {hotel_info}\n{target_hotel}: {target_info}"
        }, ignore_index=True)
    else:
        result_df = result_df.append({
            'Hotel': hotel_name,
            'Target Hotel': target_hotel,
            'Similarity Score': 'Not found',
            'Information': 'At least one of the hotels not found'
        }, ignore_index=True)

# Display the result DataFrame
result_df


,Hotel,Target Hotel,Similarity Score,Information
0,TMS Pullman Căn Hộ Cao Cấp Quy Nhơn,Grand Hyams Hotel - Quy Nhon Beach,0.995486,TMS Pullman Căn Hộ Cao Cấp Quy Nhơn: {'Name_Ho...
1,FLC City Hotel Beach Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.951027,FLC City Hotel Beach Quy Nhon: {'Name_Hotel': ...
2,FLC Sea Tower Quy Nhơn- Đông Ngô House,Grand Hyams Hotel - Quy Nhon Beach,0.996010,FLC Sea Tower Quy Nhơn- Đông Ngô House: {'Name...
3,Seaside Boutique Resort Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.965583,Seaside Boutique Resort Quy Nhon: {'Name_Hotel...
4,TMS Pullman Quy Nhơn The Maison House,Grand Hyams Hotel - Quy Nhon Beach,0.977883,TMS Pullman Quy Nhơn The Maison House: {'Name_...
5,FLC Sea Tower Qui Nhơn Apartment - Căn Hộ Sea ...,Grand Hyams Hotel - Quy Nhon Beach,0.998903,FLC Sea Tower Qui Nhơn Apartment - Căn Hộ Sea ...
6,L'amor Boutique Hotel Quy Nhơn,Grand Hyams Hotel - Quy Nhon Beach,0.926165,L'amor Boutique Hotel Quy Nhơn: {'Name_Hotel':...
7,Armin Homes Altara Residences Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.997836,Armin Homes Altara Residences Quy Nhon: {'Name...
8,Anya Premier Hotel Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.912054,Anya Premier Hotel Quy Nhon: {'Name_Hotel': 'A...
9,TMS Luxury Condotel Quy Nhơn - Ban Công View Biển,Grand Hyams Hotel - Quy Nhon Beach,0.947852,TMS Luxury Condotel Quy Nhơn - Ban Công View B...


In [130]:
min_similarity = similarity_scores.min()
max_similarity = similarity_scores.max()

# Hiển thị giá trị tối thiểu và tối đa
print("Minimum Similarity Score:", min_similarity)
print("Maximum Similarity Score:", max_similarity)
print(similarity_scores)
similarity_scores.shape

Minimum Similarity Score: -0.7727318227989447
Maximum Similarity Score: 1.0000000000000009
[[ 1.          0.85275657  0.77581681 ...  0.29868256 -0.10980886
   0.75774656]
 [ 0.85275657  1.          0.66049021 ...  0.4186436  -0.07373428
   0.84308214]
 [ 0.77581681  0.66049021  1.         ...  0.22430697 -0.08471741
   0.514411  ]
 ...
 [ 0.29868256  0.4186436   0.22430697 ...  1.          0.26393833
   0.46307291]
 [-0.10980886 -0.07373428 -0.08471741 ...  0.26393833  1.
   0.03973502]
 [ 0.75774656  0.84308214  0.514411   ...  0.46307291  0.03973502
   1.        ]]


(443, 443)